# Single analysis notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [2]:
# imports 
import sys, os.path
root_dir = (os.path.abspath(os.path.join(os.path.abspath(''), '..')))
sys.path.append(root_dir)
from scraper import stock_daily
from analyzer import Analyzer
from IPython.display import display
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import indicators as ind
import warnings
import pmdarima.arima as arm

warnings.filterwarnings('ignore')

## Settings
* Select stock ticker
* Define buy and sell strategy (buystrat and sellstrat)
* Define if you want to run stop loss and what value
* Same with profit taker

In [4]:
xls = pd.ExcelFile((os.path.abspath(os.path.join(os.path.abspath(''), '..'))) + '/Data/IWM_top600.xls')
# stop loss
sl = True
slval = 0.04
# profit taker
pt = False
ptval = 0.2
# repeteated buys
rb = False

## Running calculation

In [ ]:
chosen = pd.DataFrame()
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# print(fundTickers)
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    # print(sheet1)
    fundTickers.loc[i,["Fund/sector ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][0:]
    # print(Tickers)
pd.set_option('display.expand_frame_repr', False)    
# display(Tickers)
# I have the initialization here on purpose, to measure its impact
col = ["Ticker", "Trades", "Good trades", "Profit[%]", "Profit[$]", "Hold[%]"]
results = [None] * Tickers.columns.shape[0]
for k in range(Tickers.columns.shape[0]):
    # when I tried to initialize results as [dataframe] * 10 it created list of identical dataframes and when i changed one, it changed them all. Interesting
    results[k] = pd.DataFrame(data=np.zeros([Tickers.shape[0], len(col)]),columns=col)
# iterate over columns
i = 0 
for label, tic in Tickers.iteritems():
    # print(label)
    # print(Tickers)
    # iterate over items in sectors / indexes
    for j, val in tic.iteritems():
        # j starts at 1, instead of usual 0 => reason why there is -1 everywhere
        # analyze stock
        # print("print")
        print(j)
        try:
            stock = stock_daily(val, save=False)
            trades = Analyzer(ticker=val,data=stock.data)
            # fig=plt.figure()
            # plt.plot(stock.data["Close"], color="b")
            # plt.show()
            # adf_test = arm.ADFTest(alpha = 0.05)
            # print(adf_test.should_diff(stock.data["Close"][-100:]))
            train_0 = stock.data["Close"][-200:]
            train_1 = train_0.reset_index()
            train = train_1["Close"]
            # print(train.values)
            model = arm.auto_arima(train.values, start_p=10, start_q=10,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=20, max_q=20, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      random=False,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=False)
            n_periods = 20
            fc, confint = model.predict(n_periods=n_periods, return_conf_int=True,alpha=0.05)
            index_of_fc = np.arange(len(train), len(train)+n_periods)
            fc_series = pd.Series(fc, index=index_of_fc)
            lower_series = pd.Series(confint[:, 0], index=index_of_fc)
            upper_series = pd.Series(confint[:, 1], index=index_of_fc)
            compare = stock.data["Close"][-200:].reset_index()
            AbsReward=confint[-1, 1]
            AbsRisk=confint[-1, 0]
            BuyPrice=compare["Close"].iloc[-1]
            Risk=(BuyPrice-AbsRisk)/BuyPrice*100
            Reward=(AbsReward-BuyPrice)/BuyPrice*100
            RR=Reward/Risk
            if RR>1.5:
                chosen = chosen.append({"R/R": RR,"Ticker": stock.ticker}, ignore_index=True)                  
        except:
            results[i].loc[j-1,["Ticker"]] = val         

0
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=753.782, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=753.863, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=754.554, Time=0.05 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=755.569, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=756.873, Time=0.12 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=758.595, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=754.151, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=752.835, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=754.827, Time=0.13 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=756.800, Time=0.20 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.60 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=755.138, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=754.826, Time=0.17 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=758.800, Time=0.25 sec
 ARIMA(3,1,0)(0,0,0)[0] interce

 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=1057.878, Time=0.22 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.50 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=1058.024, Time=0.10 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=1058.905, Time=0.35 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=1059.451, Time=0.13 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 3.898 seconds
6
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=902.252, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=904.156, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=905.221, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=906.784, Time=0.07 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=908.781, Time=0.09 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=909.684, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=904.169, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=905.290, Time=0.10 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=906.957, Time=0.22 sec
 ARIMA(1,1,

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=1048.380, Time=0.10 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=1049.542, Time=0.18 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=1046.703, Time=0.12 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=1048.417, Time=0.21 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=1049.263, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=1050.152, Time=0.11 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1050.818, Time=0.12 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=1046.419, Time=0.07 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=1047.300, Time=0.16 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=1049.300, Time=0.24 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=1047.729, Time=0.11 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=1049.300, Time=0.26 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=1049.240, Time=0.13 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 2.857 seconds
14
 

 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=866.990, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=863.766, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=865.728, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=867.570, Time=0.07 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=867.550, Time=0.09 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=869.106, Time=0.11 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=863.991, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=865.681, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=868.685, Time=0.15 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=865.788, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=867.788, Time=0.06 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.48 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC

 ARIMA(3,0,1)(0,0,0)[0]             : AIC=904.345, Time=0.17 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=900.963, Time=0.30 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=inf, Time=0.12 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=905.857, Time=0.23 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=inf, Time=0.18 sec

Best model:  ARIMA(3,0,2)(0,0,0)[0]          
Total fit time: 3.876 seconds
25
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=755.439, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=757.437, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=759.249, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=761.208, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=762.930, Time=0.11 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=764.758, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=757.437, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=759.415, Time=0.26 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=761.238, Time=0.24 sec
 ARIMA(1,1,3)(0,0,

...

 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=475.457, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=475.748, Time=0.03 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=475.355, Time=0.06 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=477.347, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=479.324, Time=0.10 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=481.323, Time=0.10 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=476.131, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=476.151, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=477.345, Time=0.05 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=479.351, Time=0.32 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.35 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=475.440, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=477.405, Time=0.11 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=479.259, Time=0.26 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=481.121, Time=0.44 sec
 ARIMA(3,1,0)(0,0,0)[0] inter

 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=466.994, Time=0.22 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.55 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=466.989, Time=0.08 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=468.989, Time=0.10 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=468.975, Time=0.11 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 3.287 seconds
92
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=392.354, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=393.611, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=395.476, Time=0.07 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=397.042, Time=0.08 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=398.633, Time=0.11 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=400.595, Time=0.13 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=393.569, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=394.875, Time=0.13 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=396.875, Time=0.35 sec
 ARIMA(1,1,3)(

 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=568.186, Time=0.23 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=564.247, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=566.247, Time=0.05 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=568.241, Time=0.13 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=566.195, Time=0.05 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=568.181, Time=0.14 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.47 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=568.054, Time=0.08 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=569.066, Time=0.07 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 3.509 seconds
98
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1057.604, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1059.411, Time=0.03 sec
 ARIMA(0,1,2)(0,0,0)[0] 

 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=1090.675, Time=0.11 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=1090.556, Time=0.12 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1084.967, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=1086.000, Time=0.10 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=1087.965, Time=0.25 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=1089.835, Time=0.38 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=1092.537, Time=0.42 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=1086.954, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=1087.963, Time=0.19 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1089.830, Time=0.24 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=1088.736, Time=0.08 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=1090.624, Time=0.21 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=1091.736, Time=0.31 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=1090.724, Time=0.10 sec
 ARIMA(4,1,1)(0

 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=333.478, Time=0.30 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=336.110, Time=0.17 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=334.755, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=333.559, Time=0.19 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.50 sec


In [ ]:
# print(model.summary())


In [ ]:
# model.plot_diagnostics(figsize=(7,5))
# plt.show()

In [3]:
#Forcast one STICKER

tickerInput="CF"


stock = stock_daily(tickerInput, save=False)
trades = Analyzer(ticker=val,data=stock.data)
train_0 = stock.data["Close"][-200:]
train_1 = train_0.reset_index()
train = train_1["Close"]
model = arm.auto_arima(train.values, start_p=10, start_q=10,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=20, max_q=20, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      random=False,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=False)
# Forecast
n_periods = 20
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True,alpha=0.05)
index_of_fc = np.arange(len(train), len(train)+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
# Plot
compare = stock.data["Close"][-200:].reset_index()
plt.plot(train)
# plt.plot(compare["Close"][-30:], color='yellow')
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
plt.title("1 month "+tickerInput+" forecast - 95% confidence interval")
plt.show()
AbsReward=confint[-1, 1]
AbsRisk=confint[-1, 0]
BuyPrice=compare["Close"].iloc[-1]
Risk=(BuyPrice-AbsRisk)/BuyPrice*100
Reward=(AbsReward-BuyPrice)/BuyPrice*100
RR=Reward/Risk
print("Buy price: ",compare["Close"].iloc[-1])
print("Risk: ",Risk,"%")
print("Reward: ",Reward,"%")
print("R/R: ",RR)
print("Stop loss at < ",AbsRisk)

NameError: name 'val' is not defined

In [ ]:
# print(chosen)
# chosen.to_excel('smaRussel.xlsx')
print(chosen.sort_values(by=["R/R"]))
# print(chosen)
